## **Connecting to AZURE SQL database**

In [2]:
import pyodbc

# Connection parameters
server = ''
database = ''
username = ''
password = ''

# Create connection string
conn_str = (
    'DRIVER={SQL Server};'
    f'SERVER={server};'
    f'DATABASE={database};'
    f'UID={username};'
    f'PWD={password}'
)

try:
    # Establish connection
    conn = pyodbc.connect(conn_str)
    print("Successfully connected to database!")
    
except Exception as e:
    print(f"Error connecting to database: {str(e)}")

Successfully connected to database!


### **Inserting our tables into dataframes**

In [3]:
import pandas as pd

# SQL queries to fetch different tables
queries = {
    'properties': 'SELECT * FROM Properties',
    'visits': 'SELECT * FROM Visits',
    'agents': 'SELECT * FROM Agents',
    'clients': 'SELECT * FROM Clients',
    'sales': 'SELECT * FROM Sales'
}

# Dictionary to store dataframes
dfs = {}

try:
    # Fetch each table into a dataframe
    for table_name, query in queries.items():
        dfs[table_name] = pd.read_sql_query(query, conn)
        print(f"Successfully loaded {table_name} table")
        
    # Create individual dataframes for easier access
    properties = dfs['properties']
    visits = dfs['visits'] 
    agents = dfs['agents']
    clients = dfs['clients']
    sales = dfs['sales']
    
except Exception as e:
    print(f"Error fetching data: {str(e)}")


C:\Users\Hp\AppData\Local\Temp\ipykernel_8276\2003706028.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dfs[table_name] = pd.read_sql_query(query, conn)


Successfully loaded properties table


C:\Users\Hp\AppData\Local\Temp\ipykernel_8276\2003706028.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dfs[table_name] = pd.read_sql_query(query, conn)


Successfully loaded visits table


C:\Users\Hp\AppData\Local\Temp\ipykernel_8276\2003706028.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dfs[table_name] = pd.read_sql_query(query, conn)


Successfully loaded agents table


C:\Users\Hp\AppData\Local\Temp\ipykernel_8276\2003706028.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dfs[table_name] = pd.read_sql_query(query, conn)


Successfully loaded clients table


C:\Users\Hp\AppData\Local\Temp\ipykernel_8276\2003706028.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dfs[table_name] = pd.read_sql_query(query, conn)


Successfully loaded sales table


## **Merging tables to increase features**

In [4]:
df = sales.merge(properties, on='PropertyID', how='left')

In [5]:
agent_sales = sales.groupby('AgentID').size().reset_index(name='AgentTotalSales')
df = df.merge(agent_sales, on='AgentID', how='left')

In [6]:
client_purchases = sales.groupby('ClientID').size().reset_index(name='PreviousPurchases')
df = df.merge(client_purchases, on='ClientID', how='left')

In [7]:
df.head()

,SaleID,PropertyID,ClientID,AgentID,SaleDate,SalePrice,PropertyType,Location,Size_sqm,PriceUSD,AgentTotalSales,PreviousPurchases
0,1,891,291,70,2025-02-27,581152,Warehouse,Los Angeles,88,876734,18,1
1,2,886,1354,93,2024-07-12,801724,Retail,Miami,306,185170,28,3
2,3,372,702,43,2024-01-08,828974,Warehouse,New York,115,487971,21,1
3,4,505,1269,19,2024-12-06,1380483,Apartment,Houston,93,961902,17,2
4,5,130,280,63,2025-01-06,159459,Office,Los Angeles,431,160892,23,1


## **Started using AutoClean library**

In [8]:
from AutoClean import AutoClean
cleaned_df = AutoClean(df)

AutoClean process completed in 0.095252 seconds
Logfile saved to: d:\courses\data analaysis bootcamp\autoclean.log


In [12]:
print(cleaned_df.output)

      SaleID  PropertyID  ClientID  AgentID   SaleDate  SalePrice  \
0          1         891       291       70 2025-02-27     581152   
1          2         886      1354       93 2024-07-12     801724   
2          3         372       702       43 2024-01-08     828974   
3          4         505      1269       19 2024-12-06    1380483   
4          5         130       280       63 2025-01-06     159459   
...      ...         ...       ...      ...        ...        ...   
1995    1996         121       849       40 2024-05-24    1144968   
1996    1997        1000       707       60 2024-01-27    1338110   
1997    1998         615      1435       58 2023-09-16     855539   
1998    1999         137       330       52 2025-01-27    1111731   
1999    2000         867      1109       17 2024-10-07    1216004   

     PropertyType     Location  Size_sqm  PriceUSD  ...  \
0       Warehouse  Los Angeles        88    876734  ...   
1          Retail        Miami       306    185170  .

In [10]:
# Convert AutoClean object to dataframe and convert boolean columns to 0/1 integers
cleaned_dff = pd.DataFrame(cleaned_df.output)
bool_columns = cleaned_dff.select_dtypes(include=['bool']).columns
cleaned_dff[bool_columns] = cleaned_dff[bool_columns].astype(int)
cleaned_dff

,SaleID,PropertyID,ClientID,AgentID,SaleDate,SalePrice,PropertyType,Location,Size_sqm,PriceUSD,...,PropertyType_Apartment,PropertyType_Office,PropertyType_Retail,PropertyType_Villa,PropertyType_Warehouse,Location_Chicago,Location_Houston,Location_Los Angeles,Location_Miami,Location_New York
0,1,891,291,70,2025-02-27,581152,Warehouse,Los Angeles,88,876734,...,0,0,0,0,1,0,0,1,0,0
1,2,886,1354,93,2024-07-12,801724,Retail,Miami,306,185170,...,0,0,1,0,0,0,0,0,1,0
2,3,372,702,43,2024-01-08,828974,Warehouse,New York,115,487971,...,0,0,0,0,1,0,0,0,0,1
3,4,505,1269,19,2024-12-06,1380483,Apartment,Houston,93,961902,...,1,0,0,0,0,0,1,0,0,0
4,5,130,280,63,2025-01-06,159459,Office,Los Angeles,431,160892,...,0,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1996,121,849,40,2024-05-24,1144968,Apartment,Miami,121,350677,...,1,0,0,0,0,0,0,0,1,0
1996,1997,1000,707,60,2024-01-27,1338110,Apartment,Chicago,211,706410,...,1,0,0,0,0,1,0,0,0,0
1997,1998,615,1435,58,2023-09-16,855539,Apartment,Los Angeles,306,601960,...,1,0,0,0,0,0,0,1,0,0
1998,1999,137,330,52,2025-01-27,1111731,Apartment,Miami,75,133253,...,1,0,0,0,0,0,0,0,1,0


### **Dropping unnecessary columns and exporting our dataset**

In [ ]:
# Drop unnecessary columns
columns_to_drop = ['SaleID', 'PropertyID', 'ClientID', 'AgentID', 'SaleDate','PropertyType','Location']
cleaned_dff = cleaned_dff.drop(columns=columns_to_drop)
cleaned_dff.to_csv('cleaned_sales.csv', index=False)
cleaned_dff

,SaleID,PropertyID,ClientID,AgentID,SaleDate,SalePrice,PropertyType,Location,Size_sqm,PriceUSD,...,PropertyType_Apartment,PropertyType_Office,PropertyType_Retail,PropertyType_Villa,PropertyType_Warehouse,Location_Chicago,Location_Houston,Location_Los Angeles,Location_Miami,Location_New York
0,1,891,291,70,2025-02-27,581152,Warehouse,Los Angeles,88,876734,...,0,0,0,0,1,0,0,1,0,0
1,2,886,1354,93,2024-07-12,801724,Retail,Miami,306,185170,...,0,0,1,0,0,0,0,0,1,0
2,3,372,702,43,2024-01-08,828974,Warehouse,New York,115,487971,...,0,0,0,0,1,0,0,0,0,1
3,4,505,1269,19,2024-12-06,1380483,Apartment,Houston,93,961902,...,1,0,0,0,0,0,1,0,0,0
4,5,130,280,63,2025-01-06,159459,Office,Los Angeles,431,160892,...,0,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1996,121,849,40,2024-05-24,1144968,Apartment,Miami,121,350677,...,1,0,0,0,0,0,0,0,1,0
1996,1997,1000,707,60,2024-01-27,1338110,Apartment,Chicago,211,706410,...,1,0,0,0,0,1,0,0,0,0
1997,1998,615,1435,58,2023-09-16,855539,Apartment,Los Angeles,306,601960,...,1,0,0,0,0,0,0,1,0,0
1998,1999,137,330,52,2025-01-27,1111731,Apartment,Miami,75,133253,...,1,0,0,0,0,0,0,0,1,0
